In [ ]:
from sagemaker.hyperpod.hyperpod_manager import HyperPodManager

HyperPodManager.list_clusters(region='us-east-2')
HyperPodManager.set_context('hp-cluster-for-inf-Beta2try1', region='us-east-2')

In [2]:
from sagemaker.hyperpod.inference.config.hp_endpoint_config import CloudWatchTrigger, PrometheusTrigger, AutoScalingSpec, ModelMetrics, Metrics, FsxStorage, ModelSourceConfig, Tags, TlsConfig, ConfigMapKeyRef, FieldRef, ResourceFieldRef, SecretKeyRef, ValueFrom, EnvironmentVariables, ModelInvocationPort, ModelVolumeMount, Claims, Resources, Worker
from sagemaker.hyperpod.inference.hp_endpoint import HPEndpoint
import yaml
import time

/Library/Python/3.9/site-packages/pydantic/_internal/_fields.py:200: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


In [3]:
tls_config=TlsConfig(tls_certificate_output_s3_uri='s3://tls-bucket-inf1-beta2')

model_source_config = ModelSourceConfig(
    model_source_type='fsx',
    model_location="deepseek-1-5b",
    fsx_storage=FsxStorage(
        file_system_id='fs-0e6a92495c35a81f2'
    ),
)

environment_variables = [
    EnvironmentVariables(name="HF_MODEL_ID", value="/opt/ml/model"),
    EnvironmentVariables(name="SAGEMAKER_PROGRAM", value="inference.py"),
    EnvironmentVariables(name="SAGEMAKER_SUBMIT_DIRECTORY", value="/opt/ml/model/code"),
    EnvironmentVariables(name="MODEL_CACHE_ROOT", value="/opt/ml/model"),
    EnvironmentVariables(name="SAGEMAKER_ENV", value="1"),
]

worker = Worker(
    image='763104351884.dkr.ecr.us-east-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.4.0-tgi2.3.1-gpu-py311-cu124-ubuntu22.04-v2.0',
    model_volume_mount=ModelVolumeMount(
        name='model-weights',
    ),
    model_invocation_port=ModelInvocationPort(container_port=8080),
    resources=Resources(
            requests={"cpu": "30000m", "nvidia.com/gpu": 1, "memory": "100Gi"},
            limits={"nvidia.com/gpu": 1}
    ),
    environment_variables=environment_variables,
)

In [ ]:
fsx_endpoint = HPEndpoint(
    endpoint_name='test-endpoint-name-fsx-pysdk-07-01-1',
    instance_type='ml.g5.8xlarge',
    # use a new model_name every new deployment.
    # Older image version has a bug that causes 502 bad gateway from invoking endpoint
    model_name='deepseek15b-fsx-test-pysdk-07-01-1',
    tls_config=tls_config,
    model_source_config=model_source_config,
    worker=worker,
)

In [ ]:
fsx_endpoint.create()

In [ ]:
# poll status
t = 0
timeout = 600  # 600 seconds timeout  
interval = 15  # poll every 15 seconds

while t < timeout:
    # use refresh to fetch latest status
    fsx_endpoint.refresh()

    print('Refreshing instance status...')

    try:
        # deployment status will be available immediately
        deployment_status = fsx_endpoint.status.deploymentStatus.deploymentObjectOverallState
        if deployment_status== 'DeploymentFailed':
            print('Deployment failed!')
            break

        # endpoint status will appear be available from refresh() at some point
        endpoint_status = fsx_endpoint.status.endpoints.sagemaker.state
        if endpoint_status == 'CreationCompleted':
            print('Endpoint is available!')
            break
    except:
        pass

    time.sleep(interval)
    t += interval

if t >= timeout:
    print('Endpoint creation timed out!')

# print endpoint in yaml
def print_yaml(endpoint):
    print(yaml.dump(endpoint.model_dump(exclude_none=True)))

In [6]:
# list all endpoints
endpoint_list = HPEndpoint.list()
print_yaml(endpoint_list[0])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # list all endpoints                                                                         │
│ ❱ 2 endpoint_list = HPEndpoint.list()                                                            │
│   3 print_yaml(endpoint_list[0])                                                                 │
│   4                                                                                              │
│                                                                                                  │
│ /Users/nargokul/workspace/private-sagemaker-hyperpod-cli-staging/src/sagemaker/hyperpod/inferenc │
│ e/hp_endpoint.py:102 in list                                                                     │
│                                                                                                  │
│    99 │   │   if response and response["items"]:                                                 │
│   100 │   │   │   for item in response["items"]:                                                 │
│   101 │   │   │   │   name = item["metadata"]["name"]                                            │
│ ❱ 102 │   │   │   │   endpoints.append(cls.get(name, namespace=namespace))                       │
│   103 │   │                                                                                      │
│   104 │   │   return endpoints                                                                   │
│   105                                                                                            │
│                                                                                                  │
│ /Users/nargokul/workspace/private-sagemaker-hyperpod-cli-staging/src/sagemaker/hyperpod/inferenc │
│ e/hp_endpoint.py:114 in get                                                                      │
│                                                                                                  │
│   111 │   │   │   namespace=namespace,                                                           │
│   112 │   │   )                                                                                  │
│   113 │   │                                                                                      │
│ ❱ 114 │   │   endpoint = HPEndpoint.model_validate(response["spec"], by_name=True)               │
│   115 │   │   status = response.get("status")                                                    │
│   116 │   │   if status is not None:                                                             │
│   117 │   │   │   try:                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: model_validate() got an unexpected keyword argument 'by_name'

In [5]:
endpoint = HPEndpoint.get(name='deepseek15b-fsx-test-pysdk-07-01-1')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 endpoint = HPEndpoint.get(name='deepseek15b-fsx-test-pysdk-07-01-1')                         │
│   2                                                                                              │
│                                                                                                  │
│ /Users/nargokul/workspace/private-sagemaker-hyperpod-cli-staging/src/sagemaker/hyperpod/inferenc │
│ e/hp_endpoint.py:114 in get                                                                      │
│                                                                                                  │
│   111 │   │   │   namespace=namespace,                                                           │
│   112 │   │   )                                                                                  │
│   113 │   │                                                                                      │
│ ❱ 114 │   │   endpoint = HPEndpoint.model_validate(response["spec"], by_name=True)               │
│   115 │   │   status = response.get("status")                                                    │
│   116 │   │   if status is not None:                                                             │
│   117 │   │   │   try:                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: model_validate() got an unexpected keyword argument 'by_name'

In [ ]:
# invoke
data='{"inputs": "What is the capital of Japan?"}'

# invoke
endpoint.invoke(body=data).body.read()

In [ ]:
# delete endpoint
endpoint.delete()